In [1]:
import os, sys
sys.path.append('.')

from dataclasses import dataclass
from pathlib import Path

import draccus
import torch
import torch.distributed as dist
import tqdm
import wandb
from accelerate import PartialState
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig
from transformers.modeling_outputs import CausalLMOutputWithPast

from finetune.finetune_config import FinetuneConfig
from vla.base_prompter import PurePromptBuilder
from vla.utils import PaddedCollatorForPosePrediction, runningLoss
from vla.action_tokenizer import RLbenchPoseTokenizer
from vla.dataset import save_dataset_statistics, RLbenchCotDataset
import numpy as np
import torch.nn.functional as F


/home/lawrence/anaconda3/envs/VLA-RL/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
@dataclass
class FinetuneConfig:
    # fmt: off
    vla_path: str = "/media/lawrence/Work/checkpoints/ecot-openvla-7b-bridge"   # Path to OpenVLA model 
    vla_path_q: str = "/media/lawrence/Work/checkpoints/openvla-cot-4b"   # Path to OpenVLA model

    experiment_name: str = "Weighted_loss_2"
    dataset_name: str = "pick_described_object"                                # Name of fine-tuning dataset (e.g., `droid_wipe`)
    # data_path: Path = Path(f"./datasets/{dataset_name}/data.pt")
    train_data_path: Path = Path(f"./datasets/{dataset_name}/train_data.pt")
    test_data_path: Path = Path(f"./datasets/{dataset_name}/test_data.pt")
    run_root_dir: Path = Path("./runs")                               # Path to directory to store logs & checkpoints
    adapter_dir: Path = Path("./adapter-tmp")                     # Temporary directory for LoRA weights before fusing

    # Fine-tuning Parameters
    seed: int = 42                                                  # Random seed
    episode: int = 1
    batch_size: int = 2#16                                            # Fine-tuning batch size
    test_batch_size: int = 2
    test_limit_length: int = 30
    save_steps: int = 20#5000                                          # Interval for checkpoint saving
    learning_rate: float = 5e-5                                     # Fine-tuning learning rate
    weight_decay: float = 0.01                                       # Fine-tuning weight decay
    grad_accumulation_steps: int = 4                                # Gradient accumulation steps

    # LoRA Arguments
    use_lora: bool = True                                           # Whether to use LoRA fine-tuning
    lora_rank: int = 8#32                                             # Rank of LoRA weight matrix
    lora_dropout: float = 0.0                                       # Dropout applied to LoRA weights
    use_quantization: bool = True                                  # Whether to 4-bit quantize VLA for LoRA fine-tuning
                                                                    #   => CAUTION: Reduces memory but hurts performance
    dataset_statistics: tuple = (np.array([-0.20173775, -0.36754665,  0.81396234, -3.14153998, -0.38798628, -3.14158631,  0. ]), np.array([0.41802976, 0.45118147, 1.47966564, 3.14159215, 0.30391057, 3.14157801, 1.])) # Min-Max normalization statistics

    # Tracking Parameters
    wandb_project: str = "vla-rl"                                  # Name of W&B project to log to (use default!)
    wandb_entity: str = "lawrence-rs-lin-university-of-toronto"                           # Name of entity to log under

    # fmt: on
cfg = FinetuneConfig()

In [3]:
print(f"Fine-tuning OpenVLA Model `{cfg.vla_path}` on `{cfg.dataset_name}`")

# [Validate] Ensure GPU Available & Set Device / Distributed Context
assert torch.cuda.is_available(), "Fine-tuning assumes at least one GPU is available!"
distributed_state = PartialState()
torch.cuda.set_device(device_id := distributed_state.local_process_index)
torch.cuda.empty_cache()

# Configure Unique Experiment ID & Log Directory
exp_id = (
    f"{cfg.vla_path.split('/')[-1]}+{cfg.dataset_name}"
    f"+b{cfg.batch_size * cfg.grad_accumulation_steps}"
    f"+lr-{cfg.learning_rate}"
)
if cfg.use_lora:
    exp_id += f"+lora-r{cfg.lora_rank}+dropout-{cfg.lora_dropout}"
if cfg.use_quantization:
    exp_id += "+q-4bit"

# Start =>> Build Directories
run_dir, adapter_dir = cfg.run_root_dir / exp_id, cfg.adapter_dir / exp_id
os.makedirs(run_dir, exist_ok=True)

# Quantization Config =>> only if LoRA fine-tuning
quantization_config = None
if cfg.use_quantization:
    assert cfg.use_lora, "Quantized training only supported for LoRA fine-tuning!"
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16, bnb_4bit_quant_type="nf4", #llm_int8_skip_modules = ['projector'],
    )

# Load OpenVLA Processor and Model using HF AutoClasses
processor = AutoProcessor.from_pretrained(cfg.vla_path_q, trust_remote_code=True)
vla = AutoModelForVision2Seq.from_pretrained(
    cfg.vla_path_q,
    torch_dtype=torch.bfloat16,
    # attn_implementation="sdpa",
    quantization_config=quantization_config,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map = "auto"
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Fine-tuning OpenVLA Model `/media/lawrence/Work/checkpoints/ecot-openvla-7b-bridge` on `pick_described_object`


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/home/lawrence/anaconda3/envs/VLA-RL/lib/python3.11/site-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


In [4]:
# Device Placement =>> note that BitsAndBytes automatically handles for quantized training
if cfg.use_quantization:
    vla = prepare_model_for_kbit_training(vla)
else:
    vla = vla.to(device_id)

# [LoRA] Wrap Model w/ PEFT `LoraConfig` =>> by default we set `target_modules=all-linear`
if cfg.use_lora:
    lora_config = LoraConfig(
        r=cfg.lora_rank,
        lora_alpha=min(cfg.lora_rank, 16),
        lora_dropout=cfg.lora_dropout,
        target_modules="all-linear",
        init_lora_weights="gaussian",
    )
    vla = get_peft_model(vla, lora_config)
    vla.print_trainable_parameters()

# # Wrap VLA in PyTorch DDP Wrapper for Multi-GPU Training
# vla = DDP(vla, device_ids=[device_id], find_unused_parameters=True, gradient_as_bucket_view=True)

# Create Optimizer =>> note that we default to a simple constant learning rate!
trainable_params = [param for param in vla.parameters() if param.requires_grad]
optimizer = AdamW(trainable_params, lr=cfg.learning_rate)


trainable params: 24,400,992 || all params: 7,212,525,600 || trainable%: 0.3383


In [5]:
# Create Action Tokenizer
action_tokenizer = RLbenchPoseTokenizer(processor.tokenizer, cfg.dataset_statistics)

trainset = RLbenchCotDataset(
    cfg.train_data_path,
    action_tokenizer,
    processor.tokenizer,
    image_transform=processor.image_processor.apply_transform,
    prompt_builder_fn=PurePromptBuilder,
)

testset = RLbenchCotDataset(
    cfg.test_data_path,
    action_tokenizer,
    processor.tokenizer,
    image_transform=processor.image_processor.apply_transform,
    prompt_builder_fn=PurePromptBuilder,
)

# Create Collator and DataLoader
collator = PaddedCollatorForPosePrediction(
    processor.tokenizer.model_max_length, processor.tokenizer.pad_token_id, padding_side="right"
)

train_dataloader = DataLoader(
    trainset,
    batch_size=cfg.batch_size,
    sampler=None,
    collate_fn=collator,
    num_workers=1,  # Important =>> Set to 0 if using RLDS; TFDS rolls its own parallelism!
)
test_dataloader = DataLoader(
    testset,
    batch_size=4,
    sampler=None,
    collate_fn=collator,
    num_workers=1,  # Important =>> Set to 0 if using RLDS; TFDS rolls its own parallelism!
)



In [6]:
vla.train()
vla.gradient_checkpointing_enable()
optimizer.zero_grad()

step_idx, batch = next(enumerate(train_dataloader))

with torch.autocast("cuda", dtype=torch.bfloat16):
    vla.train()
    output: CausalLMOutputWithPast = vla(
        input_ids=batch["input_ids"].to(device_id),
        attention_mask=batch["attention_mask"].to(device_id),
        pixel_values=batch["pixel_values"].to(torch.bfloat16).to(device_id),
        labels=batch["labels"],
    )
    train_nll_loss = output.loss


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


In [18]:
processor.tokenizer.decode(batch["input_ids"][0])

"<s> \n        In: What action should the robot take to pick up the chocolate jello and place in the basket?\n        Out: \nchocolate jello POSE:<object>密死ἡ</object>\nBASKET POSE:<target>交宇ൾ</target>\nGRIPPER POSE:<gripper>∣Մ；親터助给</gripper>\nREASONING: Gripper haven't grasped the chocolate jello\nSUBTASKS: 1.Move to the chocolate jello and pick it up. 2.Move over the basket. 3.Place the chocolate jello in the basket.\nCURRENT STEP: Move to the chocolate jello and pick it up.\nACTION:<action>♀세展ữ菜ន弘</action>\n</s>"

In [7]:
# Compute Accuracy and L1 Loss for Logging
output_logits = output.logits[:, vla.vision_backbone.featurizer.patch_embed.num_patches:-1]
output_gt = batch["labels"][:, 1:].to(device_id)
action_mask, gripper_mask, object_mask, target_mask = action_tokenizer.get_mask(output_gt)

In [8]:
action_logits = output_logits[action_mask][:,action_tokenizer.action_token_begin_idx:processor.tokenizer.vocab_size].view(cfg.batch_size,-1,action_tokenizer.n_bins)
gripper_logits = output_logits[gripper_mask][:,action_tokenizer.action_token_begin_idx:processor.tokenizer.vocab_size].view(cfg.batch_size,-1,action_tokenizer.n_bins)
object_logits = output_logits[object_mask][:,action_tokenizer.action_token_begin_idx:processor.tokenizer.vocab_size].view(cfg.batch_size,-1,action_tokenizer.n_bins)
target_logits = output_logits[target_mask][:,action_tokenizer.action_token_begin_idx:processor.tokenizer.vocab_size].view(cfg.batch_size,-1,action_tokenizer.n_bins)


In [9]:
train_running_loss = runningLoss()
scaler = torch.cuda.amp.GradScaler()

In [10]:
test_action = torch.tensor([0, 0, 0.5, 0, 0, 0, 1])
processor.tokenizer(action_tokenizer(test_action))

{'input_ids': [1, 29871, 31430, 31542, 31598, 31747, 31854, 31948, 31999], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
idx = torch.tensor([31448, 31548, 31598, 31748, 31848, 31948, 31998]) - action_tokenizer.action_token_begin_idx

In [12]:
logits = torch.zeros(1,7,action_tokenizer.n_bins)

In [13]:
action_tokenizer.decode(logits.scatter_(2, idx.unsqueeze(0).unsqueeze(-1), 1))

tensor([[ 0.1112,  0.0459,  0.8173,  0.0314, -0.0386,  0.0314,  0.0000]])

In [14]:
# object
pred_object = action_tokenizer.decode(object_logits, soft = True)
gt_object = batch['target_item_poses'].to(device_id)
assert pred_object.shape == gt_object.shape, f"Object shape {pred_object.shape} != {gt_object.shape}"
object_position_loss = F.mse_loss(pred_object[:,:3], gt_object[:,:3].to(torch.float32))
object_orientation_loss = F.mse_loss(pred_object[:,3:], gt_object[:,3:].to(torch.float32))

# target
pred_target = action_tokenizer.decode(target_logits, soft = True)
gt_target = batch['basket_positions'].to(device_id)
assert pred_target.shape == gt_target.shape, f"Target shape {pred_target.shape} != {gt_target.shape}"
target_position_loss = F.mse_loss(pred_target[:,:3], gt_target[:,:3].to(torch.float32))

# gripper
pred_gripper = action_tokenizer.decode(gripper_logits, soft = True)
gt_gripper = batch['gripper_poses'].to(device_id)
assert pred_gripper.shape == gt_gripper.shape, f"Gripper shape {pred_gripper.shape} != {gt_gripper.shape}"
gripper_position_loss = F.mse_loss(pred_gripper[:,:3], gt_gripper[:,:3].to(torch.float32))
gripper_orientation_loss = F.mse_loss(pred_gripper[:,3:6], gt_gripper[:,3:6].to(torch.float32))
gripper_open_gt = torch.zeros_like(gripper_logits[:,6,-2:]).scatter_(1, gt_gripper[:,6].unsqueeze(1).to(torch.int64), 1)
gripper_open_loss = F.binary_cross_entropy_with_logits(gripper_logits[:,6,-2:], gripper_open_gt.to(torch.float32))

#action
pred_action = action_tokenizer.decode(action_logits, soft = True)
gt_action = batch['actions'].to(device_id)
assert pred_action.shape == gt_action.shape, f"Action shape {pred_action.shape} != {gt_action.shape}"
action_position_loss = F.mse_loss(pred_action[:,:3], gt_action[:,:3].to(torch.float32))
action_orientation_loss = F.mse_loss(pred_action[:,3:6], gt_action[:,3:6].to(torch.float32))
action_open_gt = torch.zeros_like(action_logits[:,6,-2:]).scatter_(1, gt_action[:,6].unsqueeze(1).to(torch.int64), 1)
action_open_loss = F.binary_cross_entropy_with_logits(action_logits[:,6,-2:], action_open_gt.to(torch.float32))

normalized_loss = train_running_loss.update(
    train_nll_loss,
    object_position_loss,
    object_orientation_loss,
    target_position_loss,
    gripper_position_loss,
    gripper_orientation_loss,
    gripper_open_loss,
    action_position_loss,
    action_orientation_loss,
    action_open_loss
)

TypeError: runningLoss.update() takes 10 positional arguments but 11 were given

In [ ]:
gripper_orientation_loss.item()

8.77499008178711

In [ ]:
normalized_loss['action_open_loss']

tensor(1., device='cuda:0', grad_fn=<DivBackward0>)

In [ ]:
normalized_loss['total_loss'].backward(retain_graph=True)

## Test

In [ ]:
# vla.eval()
# best_test_loss = float("inf")
# test_loss = 0
# test_nll_loss = []
# test_object_position_loss = []
# test_object_orientation_loss = []
# test_target_position_loss = []
# test_gripper_position_loss = []
# test_gripper_orientation_loss = []
# test_gripper_open_loss = []
# test_action_position_loss = []
# test_action_orientation_loss = []
# test_action_open_loss = []
# for batch in test_dataloader:
#     with torch.no_grad(), torch.autocast("cuda", dtype=torch.bfloat16):
#         output: CausalLMOutputWithPast = vla(
#             input_ids=batch["input_ids"].to(device_id),
#             attention_mask=batch["attention_mask"].to(device_id),
#             pixel_values=batch["pixel_values"].to(torch.bfloat16).to(device_id),
#             labels=batch["labels"],
#         )
#         test_nll_loss_ = output.loss
#         test_nll_loss.append(test_nll_loss_)

#         output_logits = output.logits[:, vla.vision_backbone.featurizer.patch_embed.num_patches:-1]
#         output_gt = batch["labels"][:, 1:].to(device_id)
#         action_mask, gripper_mask, object_mask, target_mask = action_tokenizer.get_mask(output_gt)

#         action_logits = output_logits[action_mask][:,action_tokenizer.action_token_begin_idx:processor.tokenizer.vocab_size].view(4,-1,action_tokenizer.n_bins)
#         gripper_logits = output_logits[gripper_mask][:,action_tokenizer.action_token_begin_idx:processor.tokenizer.vocab_size].view(4,-1,action_tokenizer.n_bins)
#         object_logits = output_logits[object_mask][:,action_tokenizer.action_token_begin_idx:processor.tokenizer.vocab_size].view(4,-1,action_tokenizer.n_bins)
#         target_logits = output_logits[target_mask][:,action_tokenizer.action_token_begin_idx:processor.tokenizer.vocab_size].view(4,-1,action_tokenizer.n_bins)

#         # object
#         pred_object = action_tokenizer.decode(object_logits, soft = True)
#         gt_object = batch['target_item_poses'].to(device_id)
#         assert pred_object.shape == gt_object.shape, f"Object shape {pred_object.shape} != {gt_object.shape}"
#         object_position_loss = F.mse_loss(pred_object[:,:3], gt_object[:,:3])
#         object_orientation_loss = F.mse_loss(pred_object[:,3:], gt_object[:,3:])
#         test_object_position_loss.append(object_position_loss)
#         test_object_orientation_loss.append(object_orientation_loss)

#         # target
#         pred_target = action_tokenizer.decode(target_logits, soft = True)
#         gt_target = batch['basket_positions'].to(device_id)
#         assert pred_target.shape == gt_target.shape, f"Target shape {pred_target.shape} != {gt_target.shape}"
#         target_position_loss = F.mse_loss(pred_target[:,:3], gt_target[:,:3])
#         test_target_position_loss.append(target_position_loss)

#         # gripper
#         pred_gripper = action_tokenizer.decode(gripper_logits, soft = True)
#         gt_gripper = batch['gripper_poses'].to(device_id)
#         assert pred_gripper.shape == gt_gripper.shape, f"Gripper shape {pred_gripper.shape} != {gt_gripper.shape}"
#         gripper_position_loss = F.mse_loss(pred_gripper[:,:3], gt_gripper[:,:3])
#         gripper_orientation_loss = F.mse_loss(pred_gripper[:,3:6], gt_gripper[:,3:6])
#         gripper_open_loss = F.mse_loss(pred_gripper[:,6], gt_gripper[:,6])
#         test_gripper_position_loss.append(gripper_position_loss)
#         test_gripper_orientation_loss.append(gripper_orientation_loss)
#         test_gripper_open_loss.append(gripper_open_loss)

#         #action
#         pred_action = action_tokenizer.decode(action_logits, soft = True)
#         gt_action = batch['actions'].to(device_id)
#         assert pred_action.shape == gt_action.shape, f"Action shape {pred_action.shape} != {gt_action.shape}"
#         action_position_loss = F.mse_loss(pred_action[:,:3], gt_action[:,:3])
#         action_orientation_loss = F.mse_loss(pred_action[:,3:6], gt_action[:,3:6])
#         action_open_loss = F.mse_loss(pred_action[:,6], gt_action[:,6])
#         test_action_position_loss.append(action_position_loss)
#         test_action_orientation_loss.append(action_orientation_loss)
#         test_action_open_loss.append(action_open_loss)

#         test_nll_loss = torch.stack(test_nll_loss).mean()
#         test_object_position_loss = torch.stack(test_object_position_loss).mean()
#         test_object_orientation_loss = torch.stack(test_object_orientation_loss).mean()
#         test_target_position_loss = torch.stack(test_target_position_loss).mean()
#         test_gripper_position_loss = torch.stack(test_gripper_position_loss).mean()
#         test_gripper_orientation_loss = torch.stack(test_gripper_orientation_loss).mean()
#         test_gripper_open_loss = torch.stack(test_gripper_open_loss).mean()
#         test_action_position_loss = torch.stack(test_action_position_loss).mean()
#         test_action_orientation_loss = torch.stack(test_action_orientation_loss).mean()
#         test_action_open_loss = torch.stack(test_action_open_loss).mean()
        